In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from helpers import *
import Model

In [3]:
[yb_t, input_data_t, ids_t, yb_test, input_data_test, ids_test] = pickle.load(open( "dat.p", "rb" ))
tx_tr_clean = clean_data(input_data_t)
tx_te_clean = clean_data(input_data_test)

print('Train set shape ', tx_tr_clean.shape)
print('Test set shape ', tx_te_clean.shape)

Train set shape  (250000, 30)
Test set shape  (568238, 30)


In [4]:
tx_tr_clean = np.column_stack((tx_tr_clean, yb_t))
tx_te_clean = np.column_stack((tx_te_clean, yb_test))

In [5]:
def extract_patern(tx):
    sh = tx.shape
    nan_mask = np.isnan(tx)
    type_of_missing = []
    for i in range(sh[0]):
        type_of_missing.append(''.join(['1'  if nan_mask[i,j] else '0' for j in range(sh[1])]))
    type_of_missing = set(type_of_missing)
    type_of_missing_bool = []
    for line in type_of_missing:
        type_of_missing_bool.append([True if e == '1' else False for e in line])
    return nan_mask, np.array(type_of_missing_bool)

In [6]:
nan_mask_tr, type_of_missing = extract_patern(tx_tr_clean)
print(type_of_missing)

[[False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False]
 [False False False False  True  True  True False False False False False
   True False False False False False False False False False False False
  False False  True  True  True False False]
 [ True False False False  True  True  True False False False False False
   True False False False False False False False False False False False
  False False  True  True  True False False]
 [ True False False False  True  True  True False False False False False
   True False False False False False False False False False False  True
   True  True  True  True  True False False]
 [False False False False  True  True  True False False False False False
   True False False False False False False False False False False  True
   True  True  True  True  True False False]
 [ True False False False False Fal

In [ ]:
def sort_by_pattern(tx, nan_mask, type_of_missing):
    tx_sorted = [[]]*len(type_of_missing)
    for i in range(nan_mask.shape[0]):
        for k in range(type_of_missing.shape[0]):
            if np.array_equal(nan_mask[i,:],type_of_missing[k,:]):
                tx_sorted[k]=tx_sorted[k]+[tx[i,:]]
                break
    return tx_sorted
        

In [ ]:
tx_tr_sorted = sort_by_pattern(tx_tr_clean, nan_mask_tr, type_of_missing)

In [ ]:
valid_data = []
for line in type_of_missing:
    valid = []
    for (i,test) in enumerate(type_of_missing):
        plop = test[np.invert(line)]
        if not plop.any():
            valid.append(i)
    valid_data.append(valid)
    
print(valid_data)

In [ ]:
tx_tr_sorted_grouped = []
for valid in valid_data:
    temp = []
    for i in valid:
        temp += tx_tr_sorted[i]
    tx_tr_sorted_grouped.append(temp)
    
tx_tr_sorted_grouped = [np.array(e) for e in tx_tr_sorted_grouped]

In [ ]:
models = []
for tx in tx_tr_sorted_grouped:
    model = ridge_regression()
    model.fit(tx[:,-1], tx[:,:-1])
    models.append(model)

